In [1]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
BATCH_SIZE = 48
CROP_LEN = 224

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def random_manip(img,rnd):
    if rnd == 0:
        return img
    
    elif rnd == 1:
        # gamma 0.8
        new_img = skimage.exposure.adjust_gamma(img, gamma=0.8)
        return new_img
    
    elif rnd == 2:
        # gamma 1.2
        new_img = skimage.exposure.adjust_gamma(img, gamma=1.2)
        return new_img
    
    elif rnd == 3:
        # jpeg 70
        imageio.imwrite('/tmp/quality-70.jpg', img, quality=70)
        try:
            new_img = np.array(Image.open(('/tmp/quality-70.jpg')), dtype="uint8")
            os.remove('/tmp/quality-70.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 4:
        # jpeg 90
        imageio.imwrite('/tmp/quality-90.jpg', img, quality=90)
        try:
            new_img = np.array(Image.open(('/tmp/quality-90.jpg')), dtype="uint8")
            os.remove('/tmp/quality-90.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 5:
        # 2x of original image
        new_img = scipy.misc.imresize(img, 2.0, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 6:
        new_img = scipy.misc.imresize(img, 1.5, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 7:
        new_img = scipy.misc.imresize(img, 0.5, interp='bicubic')
        return new_img
    
    elif rnd == 8:
        new_img = scipy.misc.imresize(img, 0.8, interp='bicubic')
        return new_img
    
    else:
        return img

def get_img(img_path, train_flag = True, fake_rnd = 0):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        # manip rnd
        manip_rnd = np.random.randint(16) # only half manip, 1 to 8 use manip
        #manip_rnd = 5
        #print(manip_rnd,im_array.shape)
        if manip_rnd < 7 or manip_rnd > 8 : 
            # no zoom in， 随机切出 224
            im_array = random_crop(im_array)
            # manip， 随机变化，如果放大则去中间部分
            final_img = random_manip(im_array, manip_rnd)
          
        else:
            # resize zoom out， 缩小
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image， 随机切出 224
            final_img = random_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('train',manip_rnd,final_img.shape,img_path,im_array.shape)
    else:
        # center crop for valid data
        # manip rnd
        manip_rnd = fake_rnd % 16
        if manip_rnd < 7 or manip_rnd > 8 :
            # no zoom in， 取中间部分
            im_array = center_crop(im_array)
            # manip, 随机变化，如果放大则取中间部分
            final_img = random_manip(im_array, manip_rnd)
        else:
            # resize zoom out, 先缩小图片
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image, 取中间部分
            final_img = center_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('valid',manip_rnd,final_img.shape,img_path,im_array.shape)
    
    final_img = final_img/255.0
    return final_img

# 读取测试数据中间 224
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/255.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

{'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]}


In [2]:
def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True):
    curr_idx = 0
    data_cnt = len(file_list)
    fake_rnd = 0
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        if train_flag is True:
            curr_x = np.array([get_img(p,train_flag) for p in curr_fl],dtype='float32')
        else:
            # make validation data stable
            curr_x = []
            for p in curr_fl:
                tmp_img = get_img(p,train_flag,fake_rnd)
                curr_x.append(tmp_img)
                fake_rnd += 1
                fake_rnd = fake_rnd % 16
            curr_x = np.array(curr_x,dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=42)


train_gen = data_gen(train_files, BATCH_SIZE, True)
valid_gen = data_gen(train_files, BATCH_SIZE, False)
train_step = train_data_cnt//BATCH_SIZE
valid_step = train_step

# test
for x,y in train_gen:
    print(x.shape,y.shape)
    print(x.dtype)
    print(y[:3])
    break
    
# test
for x,y in valid_gen:
    print(x.shape,y.shape)
    print(x.dtype)
    print(y[:3])
    break
    

(48, 224, 224, 3) (48, 10)
float32
[[0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]]
(48, 224, 224, 3) (48, 10)
float32
[[0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]]


In [3]:
# def model
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam


Using TensorFlow backend.


In [4]:
model_pre_p = 'best_cnn2_m.h5'
model_p = 'best_cnn2_manip.h5'

if os.path.exists(model_p):
    model = load_model(model_p)
    print('load',model_p)
else:
    model = load_model(model_pre_p)
    print('load',model_pre_p)
    
    
from keras import backend as K
K.set_value(model.optimizer.lr, 0.0001)   
#model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00001)
model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=60,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )

load best_cnn2_manip.h5
Epoch 1/60
57/57 [==============================] - 820s 14s/step - loss: 0.7550 - acc: 0.7537 - val_loss: 0.7407 - val_acc: 0.7646
Epoch 2/60
57/57 [==============================] - 744s 13s/step - loss: 0.7994 - acc: 0.7292 - val_loss: 0.7372 - val_acc: 0.7683
Epoch 3/60
57/57 [==============================] - 747s 13s/step - loss: 0.8320 - acc: 0.7193 - val_loss: 0.7222 - val_acc: 0.7701
Epoch 4/60
57/57 [==============================] - 743s 13s/step - loss: 0.7869 - acc: 0.7401 - val_loss: 0.7322 - val_acc: 0.7675
Epoch 5/60
57/57 [==============================] - 738s 13s/step - loss: 0.7708 - acc: 0.7507 - val_loss: 0.7485 - val_acc: 0.7610
Epoch 6/60
57/57 [==============================] - 743s 13s/step - loss: 0.7714 - acc: 0.7438 - val_loss: 0.7174 - val_acc: 0.7657
Epoch 7/60
57/57 [==============================] - 743s 13s/step - loss: 0.7805 - acc: 0.7361 - val_loss: 0.7237 - val_acc: 0.7716
Epoch 8/60
57/57 [==============================] - 

Epoch 32/60
57/57 [==============================] - 731s 13s/step - loss: 0.7158 - acc: 0.7654 - val_loss: 0.6971 - val_acc: 0.7803
Epoch 33/60
57/57 [==============================] - 730s 13s/step - loss: 0.7703 - acc: 0.7456 - val_loss: 0.6968 - val_acc: 0.7763
Epoch 34/60
57/57 [==============================] - 732s 13s/step - loss: 0.7578 - acc: 0.7474 - val_loss: 0.6980 - val_acc: 0.7749
Epoch 35/60
57/57 [==============================] - 734s 13s/step - loss: 0.7422 - acc: 0.7526 - val_loss: 0.7005 - val_acc: 0.7767
Epoch 36/60
57/57 [==============================] - 738s 13s/step - loss: 0.7606 - acc: 0.7504 - val_loss: 0.6985 - val_acc: 0.7756
Epoch 37/60
57/57 [==============================] - 735s 13s/step - loss: 0.7568 - acc: 0.7445 - val_loss: 0.6953 - val_acc: 0.7770
Epoch 38/60
57/57 [==============================] - 729s 13s/step - loss: 0.7456 - acc: 0.7482 - val_loss: 0.6965 - val_acc: 0.7770
Epoch 39/60
57/57 [==============================] - 733s 13s/step - 

In [5]:
best_model = load_model(model_p)
test_y = []
for img_p in test_files:
    tmp_x = get_test_img(img_p)
    tmp_y = best_model.predict(np.array([tmp_x]))[0]
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

[[  5.69187328e-02   4.52639535e-03   5.82332686e-02   6.97711483e-03
    2.02290285e-02   2.13532383e-03   1.52150333e-01   5.03682315e-01
    4.35435548e-02   1.51603967e-01]
 [  8.64548110e-06   6.14697812e-03   1.18274922e-02   5.63282229e-05
    1.47277107e-02   9.48181808e-01   7.01102708e-03   7.49591133e-03
    2.07707672e-05   4.52332571e-03]
 [  4.61791083e-03   9.44868661e-04   1.44347087e-01   5.66885807e-04
    1.06459893e-02   5.17810620e-02   6.02209926e-01   1.80817410e-01
    2.64127651e-04   3.80462664e-03]
 [  1.87744362e-08   8.98274221e-03   6.91620575e-04   5.22928610e-02
    8.60338841e-05   2.93308403e-03   1.14685821e-03   1.14818127e-03
    9.31029260e-01   1.68934674e-03]
 [  1.39736278e-09   6.51430428e-01   1.28902670e-04   4.22121771e-02
    1.68957224e-04   5.07450068e-07   1.38571306e-06   2.98924088e-01
    2.86333496e-03   4.27010097e-03]]


In [6]:
import pandas as pd
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
print(df.head())
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/s_cnn_2_224_manip.csv',index=False)

                   fname    camera
0  img_0002a04_manip.tif  iPhone-6
1  img_001e31c_unalt.tif  iPhone-6
2  img_00275cf_manip.tif  iPhone-6
3  img_0034113_unalt.tif  iPhone-6
4  img_00344b7_unalt.tif  iPhone-6
                   fname             camera
0  img_0002a04_manip.tif        LG-Nexus-5x
1  img_001e31c_unalt.tif          iPhone-4s
2  img_00275cf_manip.tif           iPhone-6
3  img_0034113_unalt.tif  Samsung-Galaxy-S4
4  img_00344b7_unalt.tif         Motorola-X


In [13]:
# TTA
def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    im_array = im_array/255.0
    img_list = []
    img_list.append(center_crop(im_array))
    img_list.append(im_array[0:CROP_LEN,0:CROP_LEN,:])
    img_list.append(im_array[0:CROP_LEN,512-CROP_LEN:512,:])
    img_list.append(im_array[512-CROP_LEN:512,0:CROP_LEN,:])
    img_list.append(im_array[512-CROP_LEN:512,512-CROP_LEN:512,:])
    return np.array(img_list)
tta_img = get_tta_img(test_files[0])
print(tta_img.shape)
tta_res = best_model.predict(tta_img)
print(tta_res.shape)
print(np.sum(tta_res,axis=0))

(5, 224, 224, 3)
(5, 10)
[ 0.06371276  0.22887008  0.39493296  0.36697608  0.14128983  0.02203164
  0.7416271   2.30726767  0.22275431  0.51053774]


In [14]:
test_y = []
for img_p in test_files:
    tmp_x = get_tta_img(img_p)
    tmp_y = best_model.predict(tmp_x)
    tmp_y = np.sum(tmp_y,axis=0)
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

[[  6.37127608e-02   2.28870079e-01   3.94932956e-01   3.66976082e-01
    1.41289830e-01   2.20316425e-02   7.41627097e-01   2.30726767e+00
    2.22754315e-01   5.10537744e-01]
 [  1.56070036e-03   7.33321160e-02   2.78607279e-01   7.34847831e-03
    1.99906573e-01   3.45597529e+00   5.40538549e-01   1.70104414e-01
    9.33147781e-03   2.63294995e-01]
 [  6.00059554e-02   5.53922728e-02   1.82980931e+00   2.00297870e-02
    1.28700703e-01   9.13145542e-02   1.86309981e+00   8.00103664e-01
    1.73491519e-02   1.34195000e-01]
 [  4.24517538e-07   1.14826098e-01   5.17186942e-03   6.30446553e-01
    9.20893915e-04   2.11770646e-02   1.34111503e-02   3.63148786e-02
    3.88548183e+00   2.92249382e-01]
 [  3.21637731e-06   3.40924215e+00   4.36363630e-02   5.50696492e-01
    7.41237728e-03   1.81990923e-04   1.57955597e-04   7.94780195e-01
    1.53702229e-01   4.01868671e-02]]


In [15]:
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/s_cnn_2_224_manip_tta.csv',index=False)

                   fname             camera
0  img_0002a04_manip.tif        LG-Nexus-5x
1  img_001e31c_unalt.tif          iPhone-4s
2  img_00275cf_manip.tif           iPhone-6
3  img_0034113_unalt.tif  Samsung-Galaxy-S4
4  img_00344b7_unalt.tif         Motorola-X
